In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/College/vision_research/code/rosetta_analysis/"

/content/drive/MyDrive/College/vision_research/code/rosetta_analysis


In [3]:
!ls

matches  rosetta_neurons


In [4]:
# !git clone https://github.com/yossigandelsman/rosetta_neurons.git

In [5]:
# import sys
# sys.path.append('/content/rosetta_neurons')

# %cd /content/rosetta_neurons

In [6]:
%cd rosetta_neurons

/content/drive/MyDrive/College/vision_research/code/rosetta_analysis/rosetta_neurons


In [7]:
#@title Requirements
!pip install -r requirements.txt
!pip install transformers
!pip install openai-clip
!pip install einops
!pip install pytorch_pretrained_biggan
!pip install Ninja
!pip install timm==0.4.12
!pip install dill

  Cloning https://github.com/openai/CLIP.git (to revision b46f5ac7587d2e1862f8b7b1573179d80dcdd620) to /tmp/pip-install-xr41h52j/clip_8f5e52cad0da40baa8819466702a4b60
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-xr41h52j/clip_8f5e52cad0da40baa8819466702a4b60
  Running command git rev-parse -q --verify 'sha^b46f5ac7587d2e1862f8b7b1573179d80dcdd620'
  Running command git fetch -q https://github.com/openai/CLIP.git b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Running command git checkout -q b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/arogozhnikov/einops.git (to revision d6f7910dc80c8413b7bb400f6f6ad24e3c5f5b0f) to /tmp/pip-install-xr41h52j/einops_51059e213d77489ebfea1d163bb0d063
  Running command git clone --filter=blob:none --quiet https://github.com/arogozhnikov/eino

In [8]:
#@title Imports
from transformers import CLIPProcessor, CLIPModel
import torch
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import clip
from PIL import Image
import requests
import torch.hub
import time
import pickle
import os
import math
import match_utils
from match_utils import matching, stats, proggan, nethook, dataset, models
device = torch.device('cuda')

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [9]:
#@title Select Parameters
gan_mode = 'styleganxl' #@param ['biggan','styleganxl','stylegan2-lsun_cat', 'stylegan2-lsun_horse', 'stylegan2-lsun_car','stylegan3-afhq']
discr_mode = 'dino_vits16' #@param ['mae', 'dino_vits16', 'dino_vits8', 'dino_vitb16', 'dino_vitb8', 'dino_resnet50', "mocov2", 'clip', 'resnet50']
batch_size = 2  #@param {type:"number"}
epochs = 800 #@param {type:"number"}
discr_path = '/content/drive/MyDrive/College/vision_research/code/pretrained_models/dino_shaders21k.pth' #@param {type:"string"}

if "stylegan2" in gan_mode or gan_mode == "stylegan3" or gan_mode == "projgan":
  classidx  = None
else:
  classidx  = 0 #@param {type:"raw"}

if gan_mode == "stylegan3-afhq":
  !gdown "https://drive.google.com/uc?export=download&id=1TWqhCvcAh2Qcu8XQMDoCKwGkqfOFpqL2"
elif gan_mode == "stylegan2-lsun_car":
  !gdown "https://drive.google.com/uc?export=download&id=1kEZG4_-hFVwEYyYwfmXGbJFu9KFSN53s"
elif gan_mode == "stylegan2-lsun_cat":
  !gdown "https://drive.google.com/uc?export=download&id=1nB5RQPrWRfcLI7yHhKKkf4gjxrJW7HCj"
elif gan_mode == "stylegan2-lsun_horse":
  !gdown "https://drive.google.com/uc?export=download&id=16SHYO836iF1AGgMps-EFXP0TxDaXYp3g"

if discr_mode == "mae":
  !gdown "https://drive.google.com/uc?export=download&id=1uJJHiQ6v8Kec9lpG0RXiyeszfGEMOW-6"

discr_modelname = "dinoshaders21k" #@param{type: "string"}
save_path = f"College/vision_research/code/rosetta_analysis/matches/{gan_mode}_{discr_modelname}_cls{classidx}_{batch_size*epochs}"
save_path = "/content/drive/MyDrive/"+save_path
if not os.path.exists(save_path):
  os.makedirs(save_path)

print("Saving to, ", save_path)

Saving to,  /content/drive/MyDrive/College/vision_research/code/rosetta_analysis/matches/styleganxl_dinoshaders21k_cls0_1600


In [10]:
import importlib
importlib.reload(models)

<module 'match_utils.models' from '/content/drive/MyDrive/College/vision_research/code/rosetta_analysis/rosetta_neurons/match_utils/models.py'>

In [11]:
discr, discr_layers = models.load_discr(discr_mode, device, discr_path)

Loading custom weights from /content/drive/MyDrive/College/vision_research/code/pretrained_models/dino_shaders21k.pth...


Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip


In [12]:
gan, gan_layers = models.load_gan(gan_mode, device)

In [20]:
!cat match.py

import argparse
from transformers import CLIPProcessor, CLIPModel
import torch
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import clip
from PIL import Image
import requests
import torch.hub
import time
import pickle
import math
import os
from match_utils import matching, stats, nethook, dataset, models

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--device", default="cuda:0", type=str)
    parser.add_argument("--save_path", default=None, type=str)
    parser.add_argument("--discr_path", default=None, type=str)  # Optional custom discriminator weights
    parser.add_argument("--gan_mode", required = True, type=str)
    parser.add_argument("--discr_mode", required=True, type=str)
    parser.add_argument("--discr_modelname", required=True, type=str)
    parser.add_argument("--batch_size", default= 10, type=int)
    parser.add_argument("--epochs", def

In [13]:
###############################################################
z_dataset, c_dataset = dataset.create_dataset(gan, gan_mode, batch_size, epochs, classidx, device)
#Run matching
matching.activ_match_gan(gan = gan, gan_layers = gan_layers,
                         discr = discr, discr_layers = discr_layers,
                         gan_mode = gan_mode,
                         discr_mode = discr_mode,
                         dataset = (z_dataset, c_dataset),
                         epochs = epochs,
                         batch_size = batch_size,
                         save_path = save_path,
                         device = device)




  0%|          | 0/800 [00:00<?, ?it/s]

Setting up PyTorch plugin "bias_act_plugin"... 

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... 

  0%|          | 0/800 [00:34<?, ?it/s]

Failed!


KeyboardInterrupt: 